# Tarea 1 <br/> CC6204 Deep Learning, Universidad de Chile  <br/> Hoja de respuestas partes 1 y 2 
## Nombre: Martin Cornejo Saavedra
Fecha sugerida para completar esta parte: 23 de marzo de 2018

# Parte 1: Funciones de activación, derivadas y función de salida

## 1a) Funciones de activación

In [0]:
# Tu código acá

def relu(T):
    S = torch.abs(T)
    return torch.div(torch.add(T, 1, S), 2)

def sig(T):
    return torch.reciprocal(1 + torch.exp(-1 * T))	

def swish(T, beta):
    return torch.mul(T, sig(torch.mul(T, beta)))

def celu(T, ):
  pass

## 1b) Derivando las funciones de activación


\begin{equation}
\frac{\partial\ \text{relu}(x)}{\partial x} = \ldots \\
\end{equation}
<br><br>

\begin{eqnarray}
\frac{\partial\ \text{swish}(x, \ldots)}{\partial x} & = & \ldots \\
\frac{\partial\ \text{swish}(x, \ldots)}{\partial \ldots} & = & \ldots \\
\end{eqnarray}
<br><br>

\begin{eqnarray}
\frac{\partial\ \text{celu}(x, \ldots)}{\partial x} & = & \ldots \\
\frac{\partial\ \text{celu}(x, \ldots)}{\partial \ldots} & = & \ldots \\
\end{eqnarray}

(Las deducciones correspondientes puedes entregarlas en otro archivo, o incluirlas directamente arriba.)

## 1c) Softmax

(La demostración puedes entregarla en otro archivo o incluirla directamente acá)

In [0]:
# Tu código acá

def softmax(T, dim=0, estable=True):
  pass

# Parte 2: Red neuronal y pasada hacia adelante (forward)

## 2a) Clase para red neuronal, 2b) Usando la GPU, 2c) Pasada hacia adelante

In [0]:
# Tu código debiera continuar así 

class FFNN():
  def __init__(self, F, l_h, l_a, C):
    pass
  
  def gpu(self):
    pass
  
  def cpu(self):
    pass
  
  def forward(self, x):
    # Usar parámetros y funciones de activación
    # El valor de retorno debiera ser y = softmax(capa_de_salida)
    pass

## 2d) Probando tu red con un modelo pre-entrenado

In [0]:
# Tu código acá

In [0]:
# Tu código visualizando los ejemplos incorrectos acá